## CRAWL DARI DETIK

### Def untuk crawling dari Portal DETIK
Note : 
- ganti User-Agent pada headers -> ke user agent browser sendiri

In [9]:
import os
import time
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from requests_html import AsyncHTMLSession  # ⬅️ ini tambahan

def detik_scraper(keywords, n_berita):
    asession = AsyncHTMLSession()
    asession.browser_args = ['--no-sandbox']

    save_dir = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/corpus/detik"
    os.makedirs(save_dir, exist_ok=True)

    async def scrape_keyword(keyword):
        print(f"\n[START] '{keyword}'")
        data_detik = []
        page = 1

        while len(data_detik) < n_berita:
            url = f'https://www.detik.com/search/searchall?query={quote(keyword)}&page={page}&result_type=relevansi'
            print(f"[INFO] Render page {page} for '{keyword}'...")

            try:
                r = await asession.get(url, timeout=20)
                await r.html.arender(timeout=20, sleep=2)
                soup = BeautifulSoup(r.html.html, 'lxml')
                links = soup.select('a.media__link')

                if not links:
                    print("[INFO] Tidak ada artikel lagi.")
                    break

                for link in links:
                    judul = link.get_text(strip=True)
                    url_berita = link['href']

                    detail_req = await asession.get(url_berita, timeout=10)
                    await detail_req.html.arender(timeout=10, sleep=1)
                    soup_detail = BeautifulSoup(detail_req.html.html, 'lxml')
                    body = soup_detail.select_one('div.read__content, div.itp_bodycontent')

                    if not body:
                        continue

                    for t in body.find_all('table', class_='linksisip'):
                        t.decompose()

                    berita = body.get_text(" ", strip=True)
                    if len(berita) > 50:
                        data_detik.append({
                            'judul': judul,
                            'berita': berita,
                            'url': url_berita
                        })
                        print(f"   [{len(data_detik)}] {judul}")

                    if len(data_detik) >= n_berita:
                        break

                page += 1

            except Exception as e:
                print(f"[ERROR] {e}")
                break

        if data_detik:
            fname = os.path.join(save_dir, f"detik_{keyword.replace(' ','_')}.csv")
            with open(fname, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader(); writer.writerows(data_detik)
            print(f"[DONE] {len(data_detik)} artikel disimpan ke {fname}")

    # Jalankan semua keyword secara berurutan (bisa paralel juga kalau mau)
    tasks = [lambda kw=kw: scrape_keyword(kw) for kw in keywords]
    asession.run(*tasks)



In [ ]:

keywords = [
    "BI rate",
    "Pertumbuhan ekonomi",
    "Inflasi",
    "Bank Indonesia",
    "Perbankan digital",
    "IHSG",
    "Pelemahan Rupiah",
    "Investasi",
    "Ekonomi Indonesia",
    "OJK",
    "Reksadana",
    "Bursa Efek Indonesia",
    "BEI",
    "Ekonomi digital"
]

detik_scraper(keywords, n_berita=10)


RuntimeError: This event loop is already running


[START] 'BI rate'
[INFO] Render page 1 for 'BI rate'...

[START] 'Pertumbuhan ekonomi'
[INFO] Render page 1 for 'Pertumbuhan ekonomi'...

[START] 'Inflasi'
[INFO] Render page 1 for 'Inflasi'...

[START] 'Bank Indonesia'
[INFO] Render page 1 for 'Bank Indonesia'...

[START] 'Perbankan digital'
[INFO] Render page 1 for 'Perbankan digital'...

[START] 'IHSG'
[INFO] Render page 1 for 'IHSG'...

[START] 'Pelemahan Rupiah'
[INFO] Render page 1 for 'Pelemahan Rupiah'...

[START] 'Investasi'
[INFO] Render page 1 for 'Investasi'...

[START] 'Ekonomi Indonesia'
[INFO] Render page 1 for 'Ekonomi Indonesia'...

[START] 'OJK'
[INFO] Render page 1 for 'OJK'...

[START] 'Reksadana'
[INFO] Render page 1 for 'Reksadana'...

[START] 'Bursa Efek Indonesia'
[INFO] Render page 1 for 'Bursa Efek Indonesia'...

[START] 'BEI'
[INFO] Render page 1 for 'BEI'...

[START] 'Ekonomi digital'
[INFO] Render page 1 for 'Ekonomi digital'...


/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/venv/lib/python3.12/site-packages/pyppeteer/util.py:29: RuntimeWarning: coroutine 'detik_scraper.<locals>.scrape_keyword' was never awaited
  gc.collect()
[INFO] Starting Chromium download.
100%|██████████| 183M/183M [01:11<00:00, 2.54Mb/s] 
[INFO] Beginning extraction
[INFO] Chromium extracted to: /home/spil/.local/share/pyppeteer/local-chromium/1181205
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id


In [ ]:
detik_scraper(keywords, n_berita=10)


RuntimeError: This event loop is already running


[START] 'BI rate'
[INFO] Render page 1 for 'BI rate'...

[START] 'Pertumbuhan ekonomi'
[INFO] Render page 1 for 'Pertumbuhan ekonomi'...

[START] 'Inflasi'
[INFO] Render page 1 for 'Inflasi'...

[START] 'Bank Indonesia'
[INFO] Render page 1 for 'Bank Indonesia'...

[START] 'Perbankan digital'
[INFO] Render page 1 for 'Perbankan digital'...

[START] 'IHSG'
[INFO] Render page 1 for 'IHSG'...

[START] 'Pelemahan Rupiah'
[INFO] Render page 1 for 'Pelemahan Rupiah'...

[START] 'Investasi'
[INFO] Render page 1 for 'Investasi'...

[START] 'Ekonomi Indonesia'
[INFO] Render page 1 for 'Ekonomi Indonesia'...

[START] 'OJK'
[INFO] Render page 1 for 'OJK'...

[START] 'Reksadana'
[INFO] Render page 1 for 'Reksadana'...

[START] 'Bursa Efek Indonesia'
[INFO] Render page 1 for 'Bursa Efek Indonesia'...

[START] 'BEI'
[INFO] Render page 1 for 'BEI'...

[START] 'Ekonomi digital'
[INFO] Render page 1 for 'Ekonomi digital'...


/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/venv/lib/python3.12/site-packages/pyppeteer/util.py:29: RuntimeWarning: coroutine 'wait' was never awaited
  gc.collect()
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
Future exception was never retrieved
fu